# Question: What are some ideas for ensuring bikes are always stocked at the most popular stations?

### Problem Framing

Two rather easy ideas are integrating an event calendar to anticipate changes in bike usage & introducing rider incentives for bike redistribution in selected stations (after all, the vast majority of bikes are moved by riders/customers and not ops!). To be able to do this, as well as to be able to run operations at all, we need to know where and when to intevene and when to refrain from doing anything. We need to know whether a station is at a particular time a big source (net trip origins) or is going to become one soon enough, while simultaneously identifying sinks (net trip destinations), since bikes moved must come from other stations.

Bike availability depends on usage, which fluctuates at ***multiple time scales***: months (temperature effects), days of week (weekday vs weekend patterns), and hours (commuter peaks). ***These scales affect the system differently***—monthly variation impacts total volume, while hourly patterns shape station behavior. Stations shift from sinks in morning rush hours to sources during evening commutes, reflecting commuters' first-mile and last-mile needs.

Some stations are both big sinks and sources simultaneously (***bidirectional demand***), even if one "personality" dominates. This means we're ***not seeking net flow of zero***—at certain hours, a balanced station may still need intervention to maintain capacity for expected demand patterns.

As redistribution depends on vans ***road constraints***, we avoided clustering stations by trip flows (high-volume pairs aren't necessarily proximate) and administrative boundaries (bike use ignores these).

### Solution Approach

We built a zone-based rebalancing model through eight steps:
1. Geographic zones: K-means clustering on lat/lon coordinates grouped the top 300 stations into 25 zones, ensuring operationally feasible redistribution routes. Zones remain static; their roles and imbalance magnitudes change over time.
2. Zone-level block detection: For each zone×month×dow, we detect time blocks by analyzing 24-hour net flow patterns (C = ends − starts). The algorithm identifies continuous periods where smoothed C remains consistently positive (supply) or negative (demand), meeting minimum duration (2 hours) and magnitude (|C| ≥ 6 bikes) thresholds.
3. Two-regime consolidation: Overnight (23:00-04:00) gets one canonical block per month×dow by summing hourly C values—any zone with |C| ≥ 6 participates. Daytime (04:00-23:00) blocks consolidate using 3-hour tolerance, merging similar boundaries into canonical windows. This reduced 4,644 fragmented blocks to 731 (~9 per month×dow).
4. Day-level zone profiling: We identify self-correcting residential zones showing morning deficits (C < -20), evening surpluses (C > +20), and quiet middays (|C| < 15). These zones balance naturally—morning's empty docks accommodate evening returns. We found 45 such combinations across 15 zones and excluded 311 block assignments.
5. Bidirectional demand adjustment: We calculate rental pressure (starts ÷ total activity) and return pressure (ends ÷ total activity) for each zone×block. Supply zones scale movable quantity by return pressure (preserving rental capacity); demand zones scale by rental pressure (acknowledging ongoing returns). This reduced movable quantities to 53% of raw |C|, preventing the model from stripping busy stations bare.
6. Sticky zones heuristic: Zones flipping roles between consecutive blocks (supply → demand or vice versa) have quantities reduced by 50%. We identified 2,737 sticky blocks (31.7%), preventing wasteful back-and-forth moves.
7. Greedy matching: We match supply to demand zones using nearest-neighbor distance (Haversine formula), moving min(supply available, demand needed) bikes per route until exhausted or max routes reached.
8. User controls: A β parameter (0.1–0.9) scales all quantities (aggressive vs conservative rebalancing), with additional filters for minimum moves and maximum routes displayed.
The interactive map lets users select month, day-of-week, and time block to view zones needing intervention, their supply/demand status, and suggested redistribution routes—grounded in 2022 patterns and operational constraints.

### Model Limitations

1. Our model uses historical 2022 data only (no real-time updates; missing traffic, terrain, precipitation, staffing data), thereby treating monthly averages as deterministic forecasts without uncertainty quantification.
2. Block consolidation creates temporal ambiguity: Consolidating zone-detected blocks (±3 hour tolerance) reduces fragmentation (4,644 → 731) but forces zones with different timing onto shared labels, obscuring precise intervention timing. Additionally, consolidated blocks sometimes overlap (e.g., 04:00-13:00, 05:00-09:00, 08:00-16:00 coexisting), creating dropdown redundancy. This is partially mitigated by focusing on top 300 Manhattan stations with synchronized demand patterns.
3. Threshold sensitivity: Arbitrary thresholds (minimum |C| = 6, residential cutoffs, sticky penalty = 50%, β = 0.1-0.9) are tuning parameters lacking principled derivation. Small changes significantly alter recommendations without systematic optimization.
4. Greedy matching suboptimality: In the absence of linear programming skills, we use nearest-neighbor heuristics—computationally simple but not guaranteed optimal for total route distance or operational efficiency.
5. Smoothing introduces temporal blur: The 3-hour centered moving average stabilizes regime detection but blurs precise demand shift timing. Sharp 9:00am transitions may appear as gradual 8:30-9:30am shifts, misaligning blocks with operational reality.

We proceeded with these limitations as way of engaging with the question's complexity.

In [1]:
# ============================================================================
# CITIBIKE REBALANCING MODEL - COMPLETE IMPLEMENTATION
# ============================================================================

import pandas as pd
import numpy as np
import warnings
import math
from sklearn.cluster import MiniBatchKMeans

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# =============================================================================
# CONFIGURATION
# =============================================================================

CSV_PATH = r"C:\Users\magia\OneDrive\Desktop\NY_Citi_Bike\2.Data\Prepared Data\nyc_2022_essential_data.csv"

# Station filtering
TOP_N = 300  # Keep top N stations by volume

# Zone clustering
TARGET_STATIONS_PER_ZONE = 12  # Approximate stations per geographic zone

# Block detection parameters
SMOOTH_WINDOW = 3      # Hours for moving average smoothing
EPS = 1.0              # Deadband threshold (treat |C| < EPS as neutral)
MIN_BLOCK_HOURS = 2    # Minimum block duration
MIN_ABS_C = 6          # Minimum |total C| to create a block

# Block consolidation
CONSOLIDATION_TOLERANCE = 3  # Hours tolerance for merging similar blocks
FORCE_OVERNIGHT_BLOCK = True  # Force a canonical overnight block
OVERNIGHT_START = 23  # Overnight block starts at 23:00 (11 PM)
OVERNIGHT_END = 4     # Overnight block ends at 04:00 (4 AM)

# Zone profiling (self-correcting zones)
RESIDENTIAL_MORNING_THRESHOLD = -20   # Morning deficit threshold
RESIDENTIAL_EVENING_THRESHOLD = 20    # Evening surplus threshold
RESIDENTIAL_MIDDAY_THRESHOLD = 15     # Max midday activity

# Sticky zones (role-flipping penalty)
FLIP_PENALTY = 0.5  # Reduce qty by 50% for zones that flip roles

print("Configuration loaded.")
print(f"Will process top {TOP_N} stations into ~{math.ceil(TOP_N/TARGET_STATIONS_PER_ZONE)} zones")

# =============================================================================
# STEP 1: LOAD AND PREPARE DATA
# =============================================================================

print("\n" + "="*80)
print("STEP 1: Loading data...")
print("="*80)

df = pd.read_csv(CSV_PATH)

# Parse datetime columns
for col in ["started_at", "ended_at", "date"]:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')

print(f"Loaded {len(df):,} trip records")
print(f"Date range: {df['started_at'].min()} to {df['started_at'].max()}")

# =============================================================================
# STEP 2: IDENTIFY TOP STATIONS AND CREATE GEOGRAPHIC ZONES
# =============================================================================

print("\n" + "="*80)
print("STEP 2: Identifying top stations and creating geographic zones...")
print("="*80)

# Get station coordinates (prefer start coords, fallback to end coords)
starts_geo = (df.dropna(subset=["start_station_name", "start_lat", "start_lng"])
              .groupby("start_station_name")[["start_lat", "start_lng"]]
              .median()
              .rename(columns={"start_lat": "lat", "start_lng": "lon"}))

ends_geo = (df.dropna(subset=["end_station_name", "end_lat", "end_lng"])
            .groupby("end_station_name")[["end_lat", "end_lng"]]
            .median()
            .rename(columns={"end_lat": "lat", "end_lng": "lon"}))

stations_all = starts_geo.combine_first(ends_geo)
stations_all.index.name = "station"

# Calculate total volume per station
starts_count = df.groupby("start_station_name").size().rename("starts")
ends_count = df.groupby("end_station_name").size().rename("ends")

station_volume = (pd.concat([starts_count, ends_count], axis=1)
                  .fillna(0)
                  .assign(total=lambda x: x.starts + x.ends)
                  .sort_values("total", ascending=False))

# Keep top N stations
top_stations = (station_volume.head(TOP_N)
                .join(stations_all, how="left")
                .dropna(subset=["lat", "lon"])
                .copy())

print(f"Selected top {len(top_stations)} stations")
print(f"Total trips covered: {top_stations['total'].sum():,.0f} ({top_stations['total'].sum()/len(df)*100:.1f}% of all trips)")

# Cluster into geographic zones using K-means
n_clusters = max(1, math.ceil(len(top_stations) / TARGET_STATIONS_PER_ZONE))
kmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=42, 
                         batch_size=256, n_init="auto")

top_stations["geo_zone"] = kmeans.fit_predict(top_stations[["lat", "lon"]])

# Remap zone IDs to sequential integers
zone_mapping = {old: new for new, old in enumerate(sorted(top_stations["geo_zone"].unique()))}
top_stations["geo_zone"] = top_stations["geo_zone"].map(zone_mapping).astype(int)

print(f"Created {top_stations['geo_zone'].nunique()} geographic zones")
print(f"Average stations per zone: {len(top_stations) / top_stations['geo_zone'].nunique():.1f}")

# Create lookup dictionaries
station_to_zone = top_stations["geo_zone"].to_dict()

# =============================================================================
# STEP 3: CALCULATE HOURLY FLOWS PER ZONE
# =============================================================================

print("\n" + "="*80)
print("STEP 3: Calculating hourly flows per zone...")
print("="*80)

# Trips STARTING from each zone (departures)
starts_hourly = (df.loc[df["start_station_name"].isin(top_stations.index)]
                 .dropna(subset=["started_at"])
                 .assign(zone=lambda x: x["start_station_name"].map(station_to_zone),
                        hour=lambda x: x["started_at"].dt.hour,
                        dow=lambda x: x["started_at"].dt.dayofweek,
                        month=lambda x: x["started_at"].dt.to_period("M").astype(str))
                 .groupby(["month", "dow", "hour", "zone"])
                 .size()
                 .rename("starts")
                 .reset_index())

# Trips ENDING at each zone (arrivals)
ends_hourly = (df.loc[df["end_station_name"].isin(top_stations.index)]
               .dropna(subset=["ended_at"])
               .assign(zone=lambda x: x["end_station_name"].map(station_to_zone),
                      hour=lambda x: x["ended_at"].dt.hour,
                      dow=lambda x: x["ended_at"].dt.dayofweek,
                      month=lambda x: x["ended_at"].dt.to_period("M").astype(str))
               .groupby(["month", "dow", "hour", "zone"])
               .size()
               .rename("ends")
               .reset_index())

# Create complete grid (all month×dow×hour×zone combinations)
all_months = sorted(set(starts_hourly["month"]) | set(ends_hourly["month"]))
all_dows = sorted(set(starts_hourly["dow"]) | set(ends_hourly["dow"]))
all_hours = range(24)
all_zones = sorted(top_stations["geo_zone"].unique())

grid = pd.MultiIndex.from_product(
    [all_months, all_dows, all_hours, all_zones],
    names=["month", "dow", "hour", "zone"]
).to_frame(index=False)

# Merge and fill missing values with 0
baseline_hourly = (grid.merge(starts_hourly, on=["month", "dow", "hour", "zone"], how="left")
                   .merge(ends_hourly, on=["month", "dow", "hour", "zone"], how="left")
                   .fillna({"starts": 0, "ends": 0}))

# Calculate net flow: C = ends - starts
# Positive C = bikes accumulating (supply/sink)
# Negative C = bikes depleting (demand/source)
baseline_hourly["C"] = baseline_hourly["ends"] - baseline_hourly["starts"]

# Smooth C to reduce noise
baseline_hourly["C_smooth"] = (baseline_hourly
    .sort_values(["month", "dow", "zone", "hour"])
    .groupby(["month", "dow", "zone"])["C"]
    .transform(lambda x: x.rolling(SMOOTH_WINDOW, center=True, min_periods=1).mean()))

print(f"Created baseline with {len(baseline_hourly):,} records")
print(f"Coverage: {len(all_months)} months × {len(all_dows)} days × 24 hours × {len(all_zones)} zones")

# =============================================================================
# STEP 4: DETECT TIME BLOCKS PER ZONE
# =============================================================================

print("\n" + "="*80)
print("STEP 4: Detecting time blocks per zone...")
print("="*80)

def detect_blocks_for_zone(df_zone):
    """
    Detect time blocks for one zone's 24-hour pattern.
    Returns list of dicts with hour_from, hour_to, C_expected, action.
    """
    df_zone = df_zone.sort_values("hour").reset_index(drop=True)
    hours = df_zone["hour"].to_numpy()
    C = df_zone["C"].to_numpy()
    C_smooth = df_zone["C_smooth"].to_numpy()
    
    # Classify hours by sign (with deadband)
    sign = np.where(C_smooth > EPS, 1, 
                    np.where(C_smooth < -EPS, -1, 0))
    
    blocks = []
    i = 0
    while i < 24:
        if sign[i] == 0:
            i += 1
            continue
        
        # Find continuous run of same sign
        current_sign = sign[i]
        j = i
        while j < 24 and sign[j] == current_sign:
            j += 1
        
        # Calculate total C over this period
        total_C = float(C[i:j].sum())
        duration = j - i
        
        # Keep block if it meets thresholds
        if duration >= MIN_BLOCK_HOURS and abs(total_C) >= MIN_ABS_C:
            blocks.append({
                "hour_from": int(hours[i]),
                "hour_to": int(hours[j-1] + 1),  # Exclusive end
                "C_expected": total_C,
                "action": "supply" if total_C > 0 else "demand"
            })
        
        i = j
    
    return blocks

# Detect blocks for each zone×month×dow
rows = []
for (m, d, z), group in baseline_hourly.groupby(["month", "dow", "zone"], sort=False):
    for block in detect_blocks_for_zone(group):
        rows.append({
            "month": m,
            "dow": int(d),
            "zone": int(z),
            **block
        })

moves_blocks_raw = pd.DataFrame(rows).sort_values(["month", "dow", "hour_from", "zone"]).reset_index(drop=True)

print(f"Detected {len(moves_blocks_raw)} raw blocks")
print(f"Average blocks per zone×month×dow: {len(moves_blocks_raw) / (len(all_months) * len(all_dows) * len(all_zones)):.1f}")

# Show sample
print("\nSample blocks:")
print(moves_blocks_raw.head(10))

# =============================================================================
# STEP 5: CONSOLIDATE SIMILAR BLOCKS
# =============================================================================

print("\n" + "="*80)
print("STEP 5: Consolidating similar blocks...")
print("="*80)

blocks_catalog_raw = (moves_blocks_raw
    .drop(columns=["zone", "C_expected", "action"])
    .drop_duplicates()
    .sort_values(["month", "dow", "hour_from"])
    .assign(block=lambda x: x["hour_from"].map("{:02d}".format) + ":00–" + 
                            x["hour_to"].map("{:02d}".format) + ":00"))

print(f"Unique blocks BEFORE consolidation: {len(blocks_catalog_raw)}")

def create_overnight_blocks(baseline_df, overnight_start=23, overnight_end=4, min_abs_c=6):
    """
    Create canonical overnight blocks directly from hourly data.
    For each month×dow×zone, sum C over overnight hours.
    """
    overnight_rows = []
    
    # Define overnight hours (wraps around midnight: 23, 0, 1, 2, 3)
    overnight_hours = [h for h in range(overnight_start, 24)] + [h for h in range(0, overnight_end)]
    
    for (m, d, z), group in baseline_df.groupby(['month', 'dow', 'zone']):
        # Get only overnight hours
        overnight_data = group[group['hour'].isin(overnight_hours)]
        
        if overnight_data.empty:
            continue
        
        # Sum C over overnight window
        total_C = overnight_data['C'].sum()
        
        # Only include if significant
        if abs(total_C) >= min_abs_c:
            overnight_rows.append({
                'month': m,
                'dow': d,
                'zone': z,
                'hour_from': overnight_start,
                'hour_to': overnight_end,
                'C_expected': total_C,
                'action': 'supply' if total_C > 0 else 'demand'
            })
    
    return pd.DataFrame(overnight_rows)

def filter_daytime_blocks(moves_blocks_df, overnight_start=23, overnight_end=4):
    """
    Remove any detected blocks that fall within overnight hours.
    Keep only blocks that are strictly in daytime (04:00-23:00).
    """
    # A block is daytime if it starts at or after overnight_end AND ends before overnight_start
    daytime_mask = (moves_blocks_df['hour_from'] >= overnight_end) & \
                   (moves_blocks_df['hour_to'] <= overnight_start)
    
    return moves_blocks_df[daytime_mask].copy()

def consolidate_daytime_blocks(daytime_df, tolerance_hours=3):
    """
    Consolidate daytime blocks using tolerance-based merging.
    """
    consolidated = []
    
    for (m, d), group in daytime_df.groupby(['month', 'dow']):
        boundaries = group[['hour_from', 'hour_to']].drop_duplicates().sort_values('hour_from')
        
        canonical_blocks = []
        used_indices = set()
        
        for idx, row in boundaries.iterrows():
            if idx in used_indices:
                continue
            
            similar_mask = (
                (abs(boundaries['hour_from'] - row['hour_from']) <= tolerance_hours) &
                (abs(boundaries['hour_to'] - row['hour_to']) <= tolerance_hours)
            )
            similar = boundaries[similar_mask]
            
            canon_from = int(similar['hour_from'].median())
            canon_to = int(similar['hour_to'].median())
            
            canonical_blocks.append((canon_from, canon_to))
            used_indices.update(similar.index)
        
        # Assign zones to canonical blocks
        for canon_from, canon_to in canonical_blocks:
            matching_zones = group[
                (abs(group['hour_from'] - canon_from) <= tolerance_hours) &
                (abs(group['hour_to'] - canon_to) <= tolerance_hours)
            ]
            
            for _, zone_row in matching_zones.iterrows():
                consolidated.append({
                    'month': m,
                    'dow': d,
                    'zone': zone_row['zone'],
                    'hour_from': canon_from,
                    'hour_to': canon_to,
                    'C_expected': zone_row['C_expected'],
                    'action': zone_row['action']
                })
    
    return pd.DataFrame(consolidated)

# Execute the two-regime approach
if FORCE_OVERNIGHT_BLOCK:
    print(f"\nCreating canonical overnight blocks ({OVERNIGHT_START:02d}:00-{OVERNIGHT_END:02d}:00)...")
    overnight_blocks = create_overnight_blocks(baseline_hourly, 
                                               overnight_start=OVERNIGHT_START,
                                               overnight_end=OVERNIGHT_END,
                                               min_abs_c=MIN_ABS_C)
    print(f"  Generated {len(overnight_blocks)} overnight zone assignments")
    
    print(f"\nFiltering daytime blocks ({OVERNIGHT_END:02d}:00-{OVERNIGHT_START:02d}:00)...")
    daytime_blocks = filter_daytime_blocks(moves_blocks_raw,
                                           overnight_start=OVERNIGHT_START,
                                           overnight_end=OVERNIGHT_END)
    print(f"  Kept {len(daytime_blocks)} daytime blocks from raw detection")
    
    print(f"\nConsolidating daytime blocks (tolerance={CONSOLIDATION_TOLERANCE}h)...")
    consolidated_daytime = consolidate_daytime_blocks(daytime_blocks,
                                                      tolerance_hours=CONSOLIDATION_TOLERANCE)
    print(f"  Consolidated to {len(consolidated_daytime)} daytime zone assignments")
    
    # Combine overnight + daytime
    moves_blocks = pd.concat([overnight_blocks, consolidated_daytime], ignore_index=True)
    moves_blocks = moves_blocks.sort_values(['month', 'dow', 'hour_from', 'zone']).reset_index(drop=True)
else:
    # No overnight enforcement, just consolidate everything
    moves_blocks = consolidate_daytime_blocks(moves_blocks_raw, 
                                              tolerance_hours=CONSOLIDATION_TOLERANCE)

# Create catalog
blocks_catalog = (moves_blocks
    .drop(columns=["zone", "C_expected", "action"])
    .drop_duplicates()
    .sort_values(["month", "dow", "hour_from"])
    .assign(block=lambda x: x["hour_from"].map("{:02d}".format) + ":00–" + 
                            x["hour_to"].map("{:02d}".format) + ":00"))

print(f"\nUnique blocks AFTER consolidation: {len(blocks_catalog)}")
print(f"Reduction: {len(blocks_catalog_raw)} → {len(blocks_catalog)} ({(1 - len(blocks_catalog)/len(blocks_catalog_raw))*100:.1f}% reduction)")

# Verify overnight blocks
if FORCE_OVERNIGHT_BLOCK:
    overnight_catalog = blocks_catalog[(blocks_catalog['hour_from'] == OVERNIGHT_START) & 
                                       (blocks_catalog['hour_to'] == OVERNIGHT_END)]
    print(f"\nOvernight blocks ({OVERNIGHT_START:02d}:00-{OVERNIGHT_END:02d}:00): {len(overnight_catalog)} found")
    if len(overnight_catalog) > 0:
        print("✓ Canonical overnight block successfully enforced")

print("\nBlocks per month×dow after consolidation (sample):")
sample_blocks = blocks_catalog.groupby(["month", "dow"])["block"].apply(list).head()
for (m, d), block_list in sample_blocks.items():
    print(f"{m} dow={d}: {len(block_list)} blocks - {block_list}")

# =============================================================================
# EXPOSE KEY OBJECTS FOR NEXT STEPS
# =============================================================================

stations_top = top_stations.rename_axis("station").reset_index()[["station", "lat", "lon", "geo_zone"]]
# moves_blocks and blocks_catalog already updated by consolidation

print("\n" + "="*80)
print("BASELINE COMPLETE")
print("="*80)
print("\nKey objects created:")
print(f"- stations_top: {len(stations_top)} stations")
print(f"- baseline_hourly: {len(baseline_hourly)} records")
print(f"- moves_blocks: {len(moves_blocks)} blocks (after consolidation)")
print(f"- blocks_catalog: {len(blocks_catalog)} unique time windows (after consolidation)")
print(f"\nAverage blocks per month×dow: {len(blocks_catalog) / (len(all_months) * len(all_dows)):.1f}")
print("\nReady for next steps:")
print("1. ✓ Block consolidation (COMPLETE)")
print("2. Day-level zone profiling")
print("3. Bidirectional demand adjustment")
print("4. Sticky zones heuristic")
print("5. Interactive map")

# =============================================================================
# STEP 6: DAY-LEVEL ZONE PROFILING (Self-Correcting Zones)
# =============================================================================

print("\n" + "="*80)
print("STEP 6: Day-level zone profiling (identify self-correcting zones)...")
print("="*80)

def identify_self_correcting_zones(moves_blocks_df, 
                                    morning_threshold=-20,
                                    evening_threshold=20,
                                    midday_threshold=15):
    """
    Identify residential zones with natural daily rhythms that don't need intervention.
    
    Criteria:
    - Morning deficit (bikes flow out): sum(C) < morning_threshold
    - Evening surplus (bikes return): sum(C) > evening_threshold
    - Low midday activity: max(|C|) < midday_threshold
    
    These zones self-correct over the day and should be excluded from recommendations.
    """
    self_correcting = []
    
    for (m, d, z), group in moves_blocks_df.groupby(['month', 'dow', 'zone']):
        # Exclude overnight block from this analysis
        daytime_blocks = group[~((group['hour_from'] == OVERNIGHT_START) & 
                                  (group['hour_to'] == OVERNIGHT_END))]
        
        if daytime_blocks.empty:
            continue
        
        # Morning blocks: ending before 12:00
        morning_blocks = daytime_blocks[daytime_blocks['hour_to'] <= 12]
        morning_C = morning_blocks['C_expected'].sum() if not morning_blocks.empty else 0
        
        # Evening blocks: starting after 16:00
        evening_blocks = daytime_blocks[daytime_blocks['hour_from'] >= 16]
        evening_C = evening_blocks['C_expected'].sum() if not evening_blocks.empty else 0
        
        # Midday blocks: between 10:00 and 16:00
        midday_blocks = daytime_blocks[(daytime_blocks['hour_from'] >= 10) & 
                                        (daytime_blocks['hour_to'] <= 16)]
        midday_max = midday_blocks['C_expected'].abs().max() if not midday_blocks.empty else 0
        
        # Check residential pattern
        is_residential = (morning_C < morning_threshold and 
                         evening_C > evening_threshold and 
                         midday_max < midday_threshold)
        
        if is_residential:
            self_correcting.append({
                'month': m,
                'dow': d,
                'zone': z,
                'morning_C': morning_C,
                'evening_C': evening_C,
                'midday_max': midday_max,
                'profile': 'residential_self_correcting'
            })
    
    return pd.DataFrame(self_correcting)

# Identify self-correcting zones
self_correcting_zones = identify_self_correcting_zones(
    moves_blocks,
    morning_threshold=RESIDENTIAL_MORNING_THRESHOLD,
    evening_threshold=RESIDENTIAL_EVENING_THRESHOLD,
    midday_threshold=RESIDENTIAL_MIDDAY_THRESHOLD
)

print(f"Found {len(self_correcting_zones)} self-correcting zone×month×dow combinations")
print(f"Unique zones flagged: {self_correcting_zones['zone'].nunique()} out of {len(all_zones)}")

if not self_correcting_zones.empty:
    print("\nSample self-correcting zones:")
    print(self_correcting_zones.head(10)[['month', 'dow', 'zone', 'morning_C', 'evening_C', 'midday_max']])
    
    # Filter moves_blocks to exclude self-correcting zones
    merge_key = ['month', 'dow', 'zone']
    moves_blocks_filtered = moves_blocks.merge(
        self_correcting_zones[merge_key + ['profile']],
        on=merge_key,
        how='left'
    )
    
    excluded_count = moves_blocks_filtered['profile'].notna().sum()
    moves_blocks = moves_blocks_filtered[moves_blocks_filtered['profile'].isna()].drop(columns=['profile'])
    
    print(f"\nExcluded {excluded_count} block assignments from self-correcting zones")
    print(f"Remaining blocks: {len(moves_blocks)}")
    
    # Update catalog
    blocks_catalog = (moves_blocks
        .drop(columns=["zone", "C_expected", "action"])
        .drop_duplicates()
        .sort_values(["month", "dow", "hour_from"])
        .assign(block=lambda x: x["hour_from"].map("{:02d}".format) + ":00–" + 
                                x["hour_to"].map("{:02d}".format) + ":00"))
    
    print(f"Updated catalog: {len(blocks_catalog)} unique time windows")
else:
    print("No self-correcting zones found with current thresholds")

print("\n✓ Day-level zone profiling COMPLETE")

# =============================================================================
# STEP 7: BIDIRECTIONAL DEMAND ADJUSTMENT
# =============================================================================

print("\n" + "="*80)
print("STEP 7: Bidirectional demand adjustment...")
print("="*80)

def add_bidirectional_metrics(moves_blocks_df, baseline_df):
    """
    Add starts/ends data to each block and calculate rental/return pressure.
    This prevents stripping busy stations of all bikes just because net flow is positive.
    """
    enriched = []
    
    for _, row in moves_blocks_df.iterrows():
        m, d, z = row['month'], row['dow'], row['zone']
        hf, ht = row['hour_from'], row['hour_to']
        
        # Get hourly data for this zone×block window
        window_data = baseline_df[
            (baseline_df['month'] == m) &
            (baseline_df['dow'] == d) &
            (baseline_df['zone'] == z) &
            (baseline_df['hour'] >= hf) &
            (baseline_df['hour'] < ht)
        ]
        
        if window_data.empty:
            # Shouldn't happen, but handle gracefully
            total_starts = 0
            total_ends = 0
        else:
            total_starts = window_data['starts'].sum()
            total_ends = window_data['ends'].sum()
        
        total_activity = total_starts + total_ends
        
        # Calculate pressure ratios (avoid division by zero)
        if total_activity > 0:
            rental_pressure = total_starts / total_activity
            return_pressure = total_ends / total_activity
        else:
            rental_pressure = 0.5
            return_pressure = 0.5
        
        enriched.append({
            'month': m,
            'dow': d,
            'zone': z,
            'hour_from': hf,
            'hour_to': ht,
            'C_expected': row['C_expected'],
            'action': row['action'],
            'starts': total_starts,
            'ends': total_ends,
            'total_activity': total_activity,
            'rental_pressure': rental_pressure,
            'return_pressure': return_pressure
        })
    
    return pd.DataFrame(enriched)

# Enrich moves_blocks with bidirectional metrics
print("Adding starts/ends data from baseline_hourly...")
moves_blocks_enriched = add_bidirectional_metrics(moves_blocks, baseline_hourly)

print(f"Enriched {len(moves_blocks_enriched)} block assignments")
print("\nSample enriched data:")
print(moves_blocks_enriched.head(10)[['month', 'dow', 'zone', 'hour_from', 'hour_to', 
                                       'C_expected', 'starts', 'ends', 'rental_pressure', 'return_pressure']])

# Calculate adjusted movable quantities
def calculate_movable_quantity(C_expected, rental_pressure, return_pressure, action):
    """
    Scale movable bikes based on bidirectional pressure.
    
    For supply zones (C > 0): Use return_pressure
      Logic: Zone has X% returns, so only X% of surplus is truly excess
    
    For demand zones (C < 0): Use rental_pressure
      Logic: Zone has X% rentals, so only X% of shortage is critical
    """
    if action == 'supply':
        # Surplus bikes, but station needs capacity for ongoing rentals
        return abs(C_expected) * return_pressure
    else:  # demand
        # Shortage, but station also has ongoing returns
        return abs(C_expected) * rental_pressure

moves_blocks_enriched['movable_raw'] = moves_blocks_enriched.apply(
    lambda r: calculate_movable_quantity(r['C_expected'], r['rental_pressure'], 
                                         r['return_pressure'], r['action']),
    axis=1
)

print("\nMovable quantities calculated (before β scaling)")
print(f"Average adjustment factor: {(moves_blocks_enriched['movable_raw'] / moves_blocks_enriched['C_expected'].abs()).mean():.2f}")

# Show examples of adjustment impact
print("\nExamples of bidirectional adjustment:")
sample = moves_blocks_enriched[moves_blocks_enriched['total_activity'] > 100].head(5)
for _, r in sample.iterrows():
    adjustment_pct = (r['movable_raw'] / abs(r['C_expected'])) * 100
    print(f"  Zone {r['zone']}, {r['action']}: C={r['C_expected']:.0f}, "
          f"starts={r['starts']:.0f}, ends={r['ends']:.0f} → "
          f"movable={r['movable_raw']:.0f} ({adjustment_pct:.0f}% of |C|)")

# Replace moves_blocks with enriched version
moves_blocks = moves_blocks_enriched.copy()

print("\n✓ Bidirectional demand adjustment COMPLETE")

# =============================================================================
# STEP 8: STICKY ZONES HEURISTIC (Forward-Looking Hedge)
# =============================================================================

print("\n" + "="*80)
print("STEP 8: Sticky zones heuristic (reduce moves for role-flipping zones)...")
print("="*80)

def identify_sticky_zones(moves_blocks_df, flip_penalty=0.5):
    """
    Identify zones that flip roles between consecutive blocks.
    Reduce their movable quantity to avoid wasteful back-and-forth moves.
    
    A zone "flips" if it's supply in block N and demand in block N+1 (or vice versa).
    """
    moves_blocks_df = moves_blocks_df.sort_values(['month', 'dow', 'zone', 'hour_from']).reset_index(drop=True)
    
    sticky_flags = []
    
    for (m, d, z), group in moves_blocks_df.groupby(['month', 'dow', 'zone']):
        group = group.sort_values('hour_from').reset_index(drop=True)
        
        for i in range(len(group)):
            is_sticky = False
            
            # Check if there's a next block for this zone
            if i < len(group) - 1:
                current_action = group.iloc[i]['action']
                next_action = group.iloc[i + 1]['action']
                
                # Zone flips if actions differ
                if current_action != next_action:
                    is_sticky = True
            
            sticky_flags.append(is_sticky)
    
    moves_blocks_df['is_sticky'] = sticky_flags
    
    # Apply penalty to sticky zones
    moves_blocks_df['movable_adjusted'] = moves_blocks_df.apply(
        lambda r: r['movable_raw'] * flip_penalty if r['is_sticky'] else r['movable_raw'],
        axis=1
    )
    
    return moves_blocks_df

# Apply sticky zones logic
print("Checking for zones that flip roles between consecutive blocks...")
moves_blocks = identify_sticky_zones(moves_blocks, flip_penalty=FLIP_PENALTY)

sticky_count = moves_blocks['is_sticky'].sum()
total_count = len(moves_blocks)
sticky_pct = (sticky_count / total_count) * 100

print(f"Found {sticky_count} sticky blocks out of {total_count} ({sticky_pct:.1f}%)")
print(f"These will have movable quantities reduced by {(1-FLIP_PENALTY)*100:.0f}%")

if sticky_count > 0:
    print("\nSample sticky zones (role-flippers):")
    sticky_sample = moves_blocks[moves_blocks['is_sticky']].head(10)
    for _, r in sticky_sample.iterrows():
        print(f"  Zone {r['zone']}, {r['month']} dow={r['dow']}, "
              f"{r['hour_from']:02d}:00-{r['hour_to']:02d}:00 ({r['action']}): "
              f"movable {r['movable_raw']:.0f} → {r['movable_adjusted']:.0f}")

print("\n✓ Sticky zones heuristic COMPLETE")

# =============================================================================
# FINAL SUMMARY
# =============================================================================

print("\n" + "="*80)
print("MODEL BUILD COMPLETE")
print("="*80)

print("\nFinal statistics:")
print(f"- Stations: {len(stations_top)}")
print(f"- Zones: {len(all_zones)}")
print(f"- Time blocks: {len(blocks_catalog)} unique windows")
print(f"- Block assignments: {len(moves_blocks)} (after all filtering)")
print(f"- Average blocks per month×dow: {len(blocks_catalog) / (len(all_months) * len(all_dows)):.1f}")

print("\nFeatures implemented:")
print("✓ Geographic zone clustering")
print("✓ Zone-level block detection")
print("✓ Block consolidation (tolerance=3h)")
print("✓ Canonical overnight block (23:00-04:00)")
print("✓ Day-level zone profiling (self-correcting zones)")
print("✓ Bidirectional demand adjustment")
print("✓ Sticky zones heuristic (role-flippers)")

print("\nReady for:")
print("- Interactive map visualization")
print("- User-controlled β parameter for intervention scaling")
print("- Greedy matching algorithm for supply→demand routes")

Configuration loaded.
Will process top 300 stations into ~25 zones

STEP 1: Loading data...
Loaded 29,838,166 trip records
Date range: 2022-01-01 00:00:13.532000 to 2022-12-31 23:58:19.206000

STEP 2: Identifying top stations and creating geographic zones...
Selected top 300 stations
Total trips covered: 32,076,043 (107.5% of all trips)
Created 25 geographic zones
Average stations per zone: 12.0

STEP 3: Calculating hourly flows per zone...
Created baseline with 50,400 records
Coverage: 12 months × 7 days × 24 hours × 25 zones

STEP 4: Detecting time blocks per zone...
Detected 7371 raw blocks
Average blocks per zone×month×dow: 3.5

Sample blocks:
     month  dow  zone  hour_from  hour_to  C_expected  action
0  2022-01    0     0          0        2        -6.0  demand
1  2022-01    0     2          0        3       -12.0  demand
2  2022-01    0     3          0        4       -13.0  demand
3  2022-01    0     5          0        3        19.0  supply
4  2022-01    0     7          0  

In [2]:
# =============================================================================
# INTERACTIVE REBALANCING MAP
# =============================================================================

import folium
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output
from shapely.geometry import MultiPoint
from shapely import concave_hull

# =============================================================================
# HELPER FUNCTIONS
# =============================================================================

def haversine_distance(lat1, lon1, lat2, lon2):
    """Calculate distance between two points in kilometers."""
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return 6371 * c  # Earth radius in km

def greedy_matching(supply_zones, demand_zones, top_k=15):
    """
    Greedy nearest-neighbor matching between supply and demand zones.
    Returns list of (supply_zone, demand_zone, bikes_moved) tuples.
    """
    if supply_zones.empty or demand_zones.empty:
        return pd.DataFrame(columns=['supply_zone', 'demand_zone', 'bikes_moved'])
    
    supply = supply_zones.copy()
    demand = demand_zones.copy()
    
    routes = []
    
    while not supply.empty and not demand.empty and supply['qty'].sum() > 0 and demand['qty'].sum() > 0:
        # Find closest supply-demand pair
        min_dist = float('inf')
        best_s_idx = None
        best_d_idx = None
        
        for s_idx, s_row in supply.iterrows():
            if s_row['qty'] <= 0:
                continue
            for d_idx, d_row in demand.iterrows():
                if d_row['qty'] <= 0:
                    continue
                dist = haversine_distance(s_row['lat'], s_row['lon'], 
                                         d_row['lat'], d_row['lon'])
                if dist < min_dist:
                    min_dist = dist
                    best_s_idx = s_idx
                    best_d_idx = d_idx
        
        if best_s_idx is None or best_d_idx is None:
            break
        
        # Move bikes
        bikes_to_move = min(supply.loc[best_s_idx, 'qty'], 
                           demand.loc[best_d_idx, 'qty'])
        
        routes.append({
            'supply_zone': supply.loc[best_s_idx, 'zone'],
            'demand_zone': demand.loc[best_d_idx, 'zone'],
            'bikes_moved': bikes_to_move
        })
        
        supply.loc[best_s_idx, 'qty'] -= bikes_to_move
        demand.loc[best_d_idx, 'qty'] -= bikes_to_move
        
        if len(routes) >= top_k:
            break
    
    return pd.DataFrame(routes).sort_values('bikes_moved', ascending=False)

def get_zone_centroids(stations_df):
    """Calculate centroid for each zone."""
    return stations_df.groupby('geo_zone')[['lat', 'lon']].mean().reset_index().rename(columns={'geo_zone': 'zone'})

def create_zone_hulls_geojson(stations_df, zones_to_show):
    """Create convex hulls for zones."""
    features = []
    for zone in zones_to_show:
        zone_stations = stations_df[stations_df['geo_zone'] == zone]
        if len(zone_stations) >= 3:
            points = [(row['lon'], row['lat']) for _, row in zone_stations.iterrows()]
            mp = MultiPoint(points)
            hull = mp.convex_hull
            features.append({
                'type': 'Feature',
                'properties': {'zone': int(zone)},
                'geometry': hull.__geo_interface__
            })
    return {'type': 'FeatureCollection', 'features': features}

def format_bikes(n):
    """Format bike count for display."""
    return f"{n/1000:.1f}k" if n >= 1000 else str(int(round(n)))

# =============================================================================
# WIDGET SETUP
# =============================================================================

# Get unique months and days
available_months = sorted(blocks_catalog['month'].unique())
available_dows = sorted(blocks_catalog['dow'].unique())
dow_labels = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

# Create widgets
w_month = widgets.Dropdown(
    options=available_months,
    value=available_months[0],
    description='Month:'
)

w_dow = widgets.Dropdown(
    options=[(dow_labels[d], d) for d in available_dows],
    value=available_dows[0],
    description='Day:'
)

w_block = widgets.Dropdown(
    options=[],
    description='Time Block:'
)

w_beta = widgets.FloatSlider(
    value=0.5,
    min=0.1,
    max=0.9,
    step=0.05,
    description='β (beta):',
    readout_format='.2f'
)

w_min_move = widgets.IntSlider(
    value=10,
    min=0,
    max=50,
    step=5,
    description='Min bikes:'
)

w_top_k = widgets.IntSlider(
    value=15,
    min=5,
    max=30,
    step=5,
    description='Max routes:'
)

output = widgets.Output()

# =============================================================================
# UPDATE FUNCTIONS
# =============================================================================

def update_block_options(*args):
    """Update block dropdown based on selected month and dow."""
    month = w_month.value
    dow = w_dow.value
    
    # Get blocks for this month×dow
    blocks_for_pair = blocks_catalog[
        (blocks_catalog['month'] == month) &
        (blocks_catalog['dow'] == dow)
    ].sort_values('hour_from')
    
    if blocks_for_pair.empty:
        w_block.options = [('No blocks available', None)]
    else:
        w_block.options = [(row['block'], f"{row['hour_from']}-{row['hour_to']}") 
                           for _, row in blocks_for_pair.iterrows()]

def draw_map(*args):
    """Draw the rebalancing map."""
    with output:
        clear_output(wait=True)
        
        if w_block.value is None:
            print("No blocks available for this selection")
            return
        
        # Parse block
        hour_from, hour_to = map(int, w_block.value.split('-'))
        
        # Get moves for this selection
        selected_moves = moves_blocks[
            (moves_blocks['month'] == w_month.value) &
            (moves_blocks['dow'] == w_dow.value) &
            (moves_blocks['hour_from'] == hour_from) &
            (moves_blocks['hour_to'] == hour_to)
        ].copy()
        
        if selected_moves.empty:
            print("No zones need intervention in this time block")
            return
        
        # Apply β scaling and min_move filter
        selected_moves['qty'] = (selected_moves['movable_adjusted'] * w_beta.value).round()
        selected_moves = selected_moves[selected_moves['qty'] >= w_min_move.value]
        
        if selected_moves.empty:
            print(f"No zones have intervention needs ≥ {w_min_move.value} bikes after β scaling")
            return
        
        # Get zone centroids
        centroids = get_zone_centroids(stations_top)
        zone_data = selected_moves.merge(centroids, on='zone', how='left')
        
        supply = zone_data[zone_data['action'] == 'supply'][['zone', 'lat', 'lon', 'qty']].copy()
        demand = zone_data[zone_data['action'] == 'demand'][['zone', 'lat', 'lon', 'qty']].copy()
        
        # Create map
        center_lat = stations_top['lat'].mean()
        center_lon = stations_top['lon'].mean()
        m = folium.Map(location=[center_lat, center_lon], zoom_start=12, tiles='cartodbpositron')
        
        # Add station markers (background)
        for _, row in stations_top.iterrows():
            folium.CircleMarker(
                [row['lat'], row['lon']],
                radius=2,
                color='#cccccc',
                fill=True,
                fill_opacity=0.3,
                opacity=0.3
            ).add_to(m)
        
        # Add zone hulls
        zones_shown = list(supply['zone']) + list(demand['zone'])
        if zones_shown:
            hulls = create_zone_hulls_geojson(stations_top, zones_shown)
            
            def style_function(feature):
                zone = feature['properties']['zone']
                action = zone_data[zone_data['zone'] == zone]['action'].iloc[0] if zone in zone_data['zone'].values else 'neutral'
                return {
                    'fillColor': '#a6dfb3' if action == 'supply' else '#f6b0b0',
                    'color': '#2e8b57' if action == 'supply' else '#c0392b',
                    'weight': 1.5,
                    'fillOpacity': 0.3
                }
            
            folium.GeoJson(hulls, style_function=style_function).add_to(m)
        
        # Greedy matching
        routes = greedy_matching(supply, demand, top_k=w_top_k.value)
        
        if not routes.empty:
            max_bikes = routes['bikes_moved'].max()
            
            for _, route in routes.iterrows():
                s_zone = centroids[centroids['zone'] == route['supply_zone']].iloc[0]
                d_zone = centroids[centroids['zone'] == route['demand_zone']].iloc[0]
                
                # Draw route line
                weight = 1 + 6 * (route['bikes_moved'] / max_bikes)
                folium.PolyLine(
                    [(s_zone['lat'], s_zone['lon']), (d_zone['lat'], d_zone['lon'])],
                    color='#555555',
                    weight=weight,
                    opacity=0.8
                ).add_to(m)
                
                # Add label at midpoint
                mid_lat = (s_zone['lat'] + d_zone['lat']) / 2
                mid_lon = (s_zone['lon'] + d_zone['lon']) / 2
                folium.Marker(
                    [mid_lat, mid_lon],
                    icon=folium.DivIcon(html=f'''
                        <div style="font-weight:700; background:white; padding:2px 6px; 
                                    border-radius:6px; opacity:0.9; font-size:11px;">
                            {format_bikes(route['bikes_moved'])}
                        </div>
                    ''')
                ).add_to(m)
        
        # Add zone center markers
        for _, row in supply.iterrows():
            folium.CircleMarker(
                [row['lat'], row['lon']],
                radius=7,
                color='#2e8b57',
                fill=True,
                fill_opacity=0.9,
                popup=f"Zone {row['zone']}: Supply {int(row['qty'])} bikes"
            ).add_to(m)
        
        for _, row in demand.iterrows():
            folium.CircleMarker(
                [row['lat'], row['lon']],
                radius=7,
                color='#c0392b',
                fill=True,
                fill_opacity=0.9,
                popup=f"Zone {row['zone']}: Demand {int(row['qty'])} bikes"
            ).add_to(m)
        
        # Add title
        month_name = pd.Period(w_month.value).strftime('%B')
        title_html = f'''
        <div style="position: fixed; top: 10px; left: 50%; transform: translateX(-50%);
                    z-index: 9999; background: rgba(255,255,255,0.95); padding: 6px 12px;
                    border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1);">
            <b>CitiBike Rebalancing — {dow_labels[w_dow.value]} {hour_from:02d}:00–{hour_to:02d}:00 ({month_name})</b>
        </div>
        '''
        m.get_root().html.add_child(folium.Element(title_html))
        
        # Fit bounds
        bounds = [[stations_top['lat'].min()-0.01, stations_top['lon'].min()-0.01],
                  [stations_top['lat'].max()+0.01, stations_top['lon'].max()+0.01]]
        m.fit_bounds(bounds)
        
        display(m)

# =============================================================================
# BIND OBSERVERS
# =============================================================================

w_month.observe(lambda change: (update_block_options(), draw_map()), names='value')
w_dow.observe(lambda change: (update_block_options(), draw_map()), names='value')
w_block.observe(draw_map, names='value')
w_beta.observe(draw_map, names='value')
w_min_move.observe(draw_map, names='value')
w_top_k.observe(draw_map, names='value')

# =============================================================================
# DISPLAY
# =============================================================================

# Initialize
update_block_options()

# Layout
controls = widgets.VBox([
    widgets.HBox([w_month, w_dow, w_block]),
    widgets.HBox([w_beta, w_min_move, w_top_k])
])

display(controls, output)
draw_map()

print("\n" + "="*80)
print("INTERACTIVE MAP READY")
print("="*80)
print("\nControls:")
print("- Month/Day/Time Block: Select when to view")
print("- β (beta): Fraction of movable bikes to actually move (0.1=conservative, 0.9=aggressive)")
print("- Min bikes: Hide zones with recommendations below this threshold")
print("- Max routes: Limit number of supply→demand routes shown")
print("\nColors:")
print("- Green zones = Supply (excess bikes)")
print("- Red zones = Demand (need bikes)")
print("- Lines = Suggested redistribution routes (thickness = quantity)")

Output()


INTERACTIVE MAP READY

Controls:
- Month/Day/Time Block: Select when to view
- β (beta): Fraction of movable bikes to actually move (0.1=conservative, 0.9=aggressive)
- Min bikes: Hide zones with recommendations below this threshold
- Max routes: Limit number of supply→demand routes shown

Colors:
- Green zones = Supply (excess bikes)
- Red zones = Demand (need bikes)
- Lines = Suggested redistribution routes (thickness = quantity)
